In [1]:
import tensorflow as tf
from tensorflow.keras import layers # type: ignore
from tensorflow.keras.preprocessing import image_dataset_from_directory # type: ignore
import pathlib
from sklearn.metrics import recall_score, accuracy_score, f1_score, precision_score, cohen_kappa_score, roc_auc_score


2025-03-24 17:24:16.507805: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-24 17:24:16.511900: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-24 17:24:16.534774: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-24 17:24:16.560729: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742837056.588905   91817 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742837056.59

In [2]:
batch_size = 32
img_height = 299
img_width  = 299

#Adjust the path according to your machine
data_dir_test  = pathlib.Path('raw_data/data_test/')


test_ds = image_dataset_from_directory(
            data_dir_test,
            image_size=(img_height, img_width),
            batch_size=batch_size)

for image_batch, labels_batch in test_ds:
  print(f"👉The shape of each test batch is {image_batch.shape}")
  print(f"  The shape of each target batch is {labels_batch.shape}")
  break

print("👀 The classes that was used for training the model are:")
for name in test_ds.class_names:
    print(f"- {name}")

num_classes = len(test_ds.class_names)


Found 451 files belonging to 7 classes.


E0000 00:00:1742837064.516081   91817 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1742837064.585204   91817 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


👉The shape of each test batch is (32, 299, 299, 3)
  The shape of each target batch is (32,)
👀 The classes that was used for training the model are:
- cataract
- degeneration
- diabets
- glaucoma
- hypertension
- myopia
- normal


In [3]:
from tensorflow.keras import layers

#Normalization of the image tensors
normalization_layer =   layers.Rescaling(1./255)
normalized_test_ds  =  test_ds.map(lambda x, y: (normalization_layer(x), y))

# Let's test our model

In [4]:
#### # Load the model
model = model = tf.keras.models.load_model("raw_data/model_Xception-02.keras")


In [5]:
print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 149, 149,  │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 149, 149,  │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 147, 147,  │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 147, 147,  │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 147, 147,  │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 147, 147,  │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 147, 147,  │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 147, 147,  │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 147, 147,  │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 147, 147,  │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 74, 74,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 74, 74,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 74, 74,    │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 74, 74,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 74, 74,    │          0 │ add[0][0]       

 Total params: 27,177,534 (103.67 MB)

 Trainable params: 2,105,351 (8.03 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

 Optimizer params: 4,210,703 (16.06 MB)

None


In [6]:
y_pred_prob = model.predict(normalized_test_ds)

15/15 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step


In [7]:
import numpy as np

y_true = np.concatenate([y.numpy() for _, y in normalized_test_ds], axis=0)
y_pred = np.argmax(y_pred_prob, axis=1)

2025-03-24 17:24:59.720076: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
test_ds.class_names

['cataract',
 'degeneration',
 'diabets',
 'glaucoma',
 'hypertension',
 'myopia',
 'normal']

# AUC for multiclass

In [9]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

encoder = OneHotEncoder(sparse_output=False)
y_true_one_hot = encoder.fit_transform(np.array(y_true).reshape(-1, 1))
y_true_one_hot

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [10]:
n_classes = y_true_one_hot.shape[1]  # Number of classes
auc_scores = []

for i in range(n_classes):
    auc = roc_auc_score(y_true_one_hot[:, i], np.array(y_pred_prob)[:, i], multi_class='ovo')
    auc_scores.append(auc)

print(f"AUC scores for each class: {auc_scores}")

AUC scores for each class: [np.float64(0.5713124746038195), np.float64(0.49612403100775193), np.float64(0.5382643188854489), np.float64(0.5330578512396694), np.float64(0.3133786848072562), np.float64(0.5605594046702592), np.float64(0.5272827412565404)]


# See other metrics

In [19]:

results = [['class','accuracy', 'recall', 'f1_score', 'precision', "roc_auc"] ]
for class_id in range(7):
    class_y_true = [1 if label == class_id else 0 for label in y_true]
    class_y_pred = [1 if pred  == class_id else 0 for pred in y_pred ]
    
    class_recall    = recall_score(class_y_true, class_y_pred, average = 'macro')

    class_accuracy  = accuracy_score(class_y_true, class_y_pred)
    
    class_f1        = f1_score(class_y_true, class_y_pred, average = 'macro')

    class_precision = precision_score(class_y_true, class_y_pred, average = 'macro')

    class_auc = auc_scores[class_id]
    #class_auc = roc_auc_score(class_y_true, class_y_pred, average = 'macro')

    results.append([test_ds.class_names[class_id], class_accuracy, class_recall, class_f1, class_precision, class_auc])

print("{:<20} {:<10} {:<10}  {:<10} {:<10} {:<10}".format(*results[0]))

for row in results[1:]:
    print("{:<20} {:<10.2f} {:<10.2f}  {:<10.2f} {:<10.2f} {:<10.2f}".format(*row))


class                accuracy   recall      f1_score   precision  roc_auc   
cataract             0.90       0.52        0.51       0.51       0.57      
degeneration         0.91       0.52        0.52       0.52       0.50      
diabets              0.71       0.51        0.44       0.57       0.54      
glaucoma             0.78       0.54        0.49       0.51       0.53      
hypertension         0.98       0.50        0.49       0.49       0.31      
myopia               0.92       0.51        0.51       0.51       0.56      
normal               0.53       0.53        0.52       0.53       0.53      


Given that our model has a high accuracy for amolst all classes, it is able to classify some instances. However, it's important to consinder that the dataset is imbalanced. When we look at the results for other metrics, we can see that the model is very close to the baseline. 

# We don't need to upload the test dataset to the clouds. We can create a dict by hand.

In [12]:
classes_dict = {0: 'cataract', 1 :'degeneration', 2:'diabets', 3: 'glaucoma', 4: 'hypertension', 5:'myopia', 6:'normal'}

In [13]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load an image and resize it to the model's expected input size (e.g., 299x299 for Xception)
image = load_img("raw_data/data_test/glaucoma/1365_right.jpg", target_size=(299, 299))  # glaucoma image
image_array_norm = img_to_array(image) / 255.0  # Normalize to [0, 1]
image_array = np.expand_dims(image_array_norm, axis=0)  # Add batch dimension

In [14]:
pred = model.predict(image_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ -1s -1337264us/step


In [15]:
pred

array([[0.21233006, 0.03225471, 0.08776455, 0.4249425 , 0.00913992,
        0.00187595, 0.23169224]], dtype=float32)

In [16]:
pred_id = np.argmax(pred, axis=1)[0]

In [17]:
classes_dict[pred_id]

'glaucoma'